In [185]:
# input n and T
# The gird n*n and run T times 
# run T the number run experiment
# p the probability of open the site


# Define UnionFInd

In [186]:
class UnionFind():
    def __init__(self, n):
        self.parent = list(range(n)) # init parent from 0 to n 
        
    def find(self, x):
        while self.parent[x] != x:
            x = self.parent[x]
        return x
    def union(self, i,j):
        self.parent[self.find(i)] = self.find(j)

In [187]:
# Initialize the UnionFind data structure with 10 elements
n = 10
uf = UnionFind(n)

# Perform some union operations
uf.union(1, 2)
uf.union(2, 4)
# Perform another union operation
uf.union(5, 6)
uf.union(4, 5)

In [188]:
# check some data
print("find of 2", uf.find(2))
print("find of 5", uf.find(5))
print(uf.parent)

find of 2 6
find of 5 6
[0, 2, 4, 3, 6, 6, 6, 7, 8, 9]


# Define percolison class

In [189]:
# public class Percolation {

#     // creates n-by-n grid, with all sites initially blocked
#     public Percolation(int n)

#     // opens the site (row, col) if it is not open already
#     public void open(int row, int col)

#     // is the site (row, col) open?
#     public boolean isOpen(int row, int col)

#     // is the site (row, col) full?
#     public boolean isFull(int row, int col)

#     // returns the number of open sites
#     public int numberOfOpenSites()

#     // does the system percolate?
#     public boolean percolates()

#     // test client (optional)
#     public static void main(String[] args)
# }
import numpy as np 
class Percolation():
    def __init__(self,n):
        # define percolation
        self.percolation = [[0]*n for _ in range(n)]
        self.uf  = UnionFind(n*n + 2)
        self.n  = n
        # the table begin from i , j = 0 , 0  with id =0 to i , j = n n with id = n * n
        # id = n* i + (j +1 )
        # id2ij = {}
        # ij2id = {}
        # for i in range(1 , n + 1):
        #     for j in range(1 , n + 1):
        #         id2ij[id]["i"] = i
        #         id2ij[id]["j"] = j
        
        self.vir_point_1 = 0
        self.vir_point_2 = n + 1
        #virtural point is 0 and n+1
        for j in range(n):
            # print("id of ij",self.id_of_ij(0,j) )
            self.uf.union(self.vir_point_1 , self.id_of_ij(0,j))
            self.uf.union(self.vir_point_2 , self.id_of_ij(n-1,j))


    def id_of_ij(self, i , j):
        return (self.n * i) + (j +  1) 


        
    def open(self,i,j):
        self.percolation[i][j] = 1
        
    def isOpen(self,i,j):
        return True if self.percolation[i][j] == 1 else False
    
    def isFull(self,i,j):
        if self.uf.find(self.id_of_ij(i,j)) == self.uf.find(0):
            return True
        else: 
            return False
    def numberOfOpenSites(self):
        total_sum = sum(sum(row) for row in self.percolation)
        return total_sum
    def percolates(self):
        if self.uf.find(self.vir_point_1) == self.uf.find(self.vir_point_2):
            return True
        else:
            return False 


In [190]:
import random
# test class
n = 5
per = Percolation(n)
# random assign 1 to array with montecalo method p
# import random
p = 0.6
for i in range(n):
    for j in range(n):
        # print()
        per.percolation[i][j] = random.choices([0, 1], weights=[1 - p, p], k=1)[0]
per.percolation

[[0, 1, 1, 1, 1],
 [0, 0, 1, 1, 0],
 [1, 1, 0, 1, 0],
 [0, 1, 0, 1, 1],
 [0, 0, 1, 0, 0]]

In [191]:
print(list(range(n*n+2)))
print(per.uf.parent)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
[1, 2, 3, 4, 5, 5, 21, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 25, 26]


In [193]:
print(per.id_of_ij(3,1))
print(per.isFull(0,1))

print(per.uf.find(17))
print(per.uf.find(0))

17
True
17
5
